In [1]:
#Junming Chen


import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics

%matplotlib inline


In [2]:
#read the 2016 Presidential election tweets and assigned column names. 

tweet_cols = ['Name', 'ScreenName', 'UserID', 'FollowersCount', 'FriendsCount','Location', 'Description', 'CreatedAt', 'StatusID', 'Language', 'Place', 'RetweetCount','FavoriteCount', 'Text']
tweets = pd.read_table('clinton_trump_tweets.txt', header=None, names=tweet_cols ,encoding= "ISO-8859-1")
pd.set_option('display.max_colwidth', -1)
#Output the first five rows
tweets.head()

,Name,ScreenName,UserID,FollowersCount,FriendsCount,Location,Description,CreatedAt,StatusID,Language,Place,RetweetCount,FavoriteCount,Text
0,Cebel,Cebel6,1519696717,132,263,"Little Rock, Arkansas",Arkansas Razorback Fan Just trying to be #Uncommon one 1-0 day at a time.,Sat Oct 29 08:10:06 EEST 2016,792232017094119425,en,NaN,0,1,@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
1,Cookie,Cookiemuffen,109945090,2154,2034,The American South,Got married after college. I don't regret starting a family instead of grad school. Proud Deplorable,Wed Oct 26 18:44:08 EEST 2016,791304413923213312,en,NaN,1937,0,RT @wikileaks: New poll puts Pirate Party on course to win Iceland's national elections on Saturday. https://t.co/edTqjeJaQ6
2,nolaguy,nolaguy_phd,1450086582,797,1188,NaN,"An LSU Ph.D student living in New Orleans, trying to find a second act.",Sat Oct 29 21:53:29 EEST 2016,792439227090767872,en,NaN,0,0,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.
3,Mark Hager,marksnark,167177185,204,448,Pittsburgh,"Hip, trendy, smart, funny, fit, lobbyist. U? Boilerplate: these thoughts are my own, not anyone else's. Hmmmkay?",Wed Oct 26 00:33:20 EEST 2016,791029904733331457,en,NaN,891,0,RT @LOLGOP: ACA needs fixes but know da facts: *70% can get covered in marketplaces for under $75/month *Hikes affect 3% *GOP will uninsu
4,Capitalist Creations,aaronjhoddinott,1191022351,775,154,Canada,"Entrepreneur, startup investor, political junkie, free market supporter, beer connoisseur, dad and dog lover. Also a golf enthusiast despite my lack of skill.",Fri Oct 28 05:05:10 EEST 2016,791823089700962304,en,NaN,7,0,RT @FastCompany: Alphabet shares soar on better-than-expected earnings as mobile video strategy pays off https://t.co/bokbXngMJt https://t.


In [3]:
#Drop Unused Columns
tweets= tweets.drop(columns=['Name','ScreenName','FollowersCount','FriendsCount', 'Description','CreatedAt','StatusID','Language','Place','RetweetCount','FavoriteCount'])


In [4]:
#Drop Retweets

tweets = tweets.drop(tweets[(tweets.Text.str.startswith("RT"))].index)

tweets.head(10)

,UserID,Location,Text
0,1519696717,"Little Rock, Arkansas",@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
2,1450086582,NaN,@gaystoner821 I think New Orleans spoiled me with food. I need to try and branch out in BR.
6,106568768,"Dallas, TX",#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv
7,17101060,The Universe,"Hacked e-mails show Clinton campaigns fears about Sanders | https://t.co/WMyCHuCDIc The Philippine Star (PhilippineStar) October 28, 2"
10,16818809,Long Island and beyond...,Hulk smash!
12,163712593,"Portland, Oregon","Well, I'm glad you got a good haircut, America's in the toilet. Is this not a conversation @Haylie_Bre and I woul https://t.co/6dVIIKZcgv"
13,2447279666,USA,@WayneDupreeShow #climatechange is the democrats' attempt to play God
18,231921777,"Milwaukee, Wisconsin",OH MY GOD. TEARS. https://t.co/8odjnHZwRK
20,235657662,On An Ultra Light Beams,"no lie, I been waiting for tony's story https://t.co/S5i4ESNilb"
21,201499452,New York City,How a dot-com millionaire became a social media crusader against solar amendment: It was a violation of Googl... https://t.co/ychypPZpKd


In [5]:
#Drop Null Values in Location

tweets = tweets.dropna(subset=['Location'])

In [6]:
#Remove Users with less than 20 tweets
#Use grouby and count to retrive amounts of tweets for each individual UserID form database
top_users = tweets.groupby('UserID').Text.count().sort_values(ascending=False)
#Include Users with only 20 tweet or more
top_users = top_users[top_users>=20]
top_users.head()

UserID
1971604699    3018
15501305      2493
17197477      2321
1732989216    2210
23719043      2133
Name: Text, dtype: int64

In [7]:
#Refresh Dataframe with UserID thats have 20 tweets or more.
#Using isin,if the userID is in the Top_Users list 
tweets =  tweets[tweets['UserID'].isin(top_users.index.tolist())]
tweets.head(10)

,UserID,Location,Text
0,1519696717,"Little Rock, Arkansas",@NWAJimmy I've read it now though brother. Was pretty spot on Lots of bright spots but a lot to work on. Exactly as an exhibition should be!
6,106568768,"Dallas, TX",#infosec #Intel #ACM #IEEE Impacts Haswell microarch. Paper proposes mitigations that could prevent BTB-based side https://t.co/DW6vgRAPrv
7,17101060,The Universe,"Hacked e-mails show Clinton campaigns fears about Sanders | https://t.co/WMyCHuCDIc The Philippine Star (PhilippineStar) October 28, 2"
10,16818809,Long Island and beyond...,Hulk smash!
12,163712593,"Portland, Oregon","Well, I'm glad you got a good haircut, America's in the toilet. Is this not a conversation @Haylie_Bre and I woul https://t.co/6dVIIKZcgv"
13,2447279666,USA,@WayneDupreeShow #climatechange is the democrats' attempt to play God
18,231921777,"Milwaukee, Wisconsin",OH MY GOD. TEARS. https://t.co/8odjnHZwRK
20,235657662,On An Ultra Light Beams,"no lie, I been waiting for tony's story https://t.co/S5i4ESNilb"
21,201499452,New York City,How a dot-com millionaire became a social media crusader against solar amendment: It was a violation of Googl... https://t.co/ychypPZpKd
22,111679943,"Oak Brook, IL",@tansleyemiley69 Drop by anytime. Well wait for you here. :)


In [8]:
#Groupby Userid and their Text
newtweets = tweets.groupby(['UserID', 'Location'], as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ' '.join(x))
#newtweets['Text'] = newtweets['Text'].apply(lambda x: re.split('https:\/\/.*', str(x))[0])
newtweets.set_index('UserID', inplace=True)
newtweets.head()

Location  \
UserID                          
1737    In my head              
2391    San Francisco           
2426    SF                      
11569   Jet Life Stadium        
14763   Portland, Oregon, USA   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [9]:
#Read Ground Truth Data
tweet_cols2 = ['UserID', 'Supporter']
groundtruth = pd.read_table('clinton_trump_user_classes.txt',names=tweet_cols2, encoding ="ISO-8859-1")

In [10]:
#Merge 2 Data Frame
merge_tweets = pd.merge(newtweets, groundtruth, on='UserID')
merge_tweets.head()

UserID               Location  \
0  1737    In my head              
1  2391    San Francisco           
2  2426    SF                      
3  11569   Jet Life Stadium        
4  14763   Portland, Oregon, USA   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [11]:
#Delete Everything in Text except @ or #
def gethash(list):
    newstr=""
    for c in list:
        if '#' in c or '@' in c:
            newstr= newstr + " " + c
    return newstr

merge_tweets['Text'] = merge_tweets.Text.str.split(' ').apply(gethash)

#merge_tweets.head()
test = pd.Series(merge_tweets['Text'].str.cat(sep=' ').split())
num = test.value_counts()
num

@YouTube              19912
@realDonaldTrump      15891
@HillaryClinton       12234
@                     6906 
@FoxNews              5456 
#WorldSeries          5423 
@CNN                  4485 
@megynkelly           4172 
#NowPlaying           4064 
#PJNET                3907 
#quote                3888 
#Trump                3859 
#Now                  3438 
#MAGA                 3424 
0                     3325 
#COSProject           3120 
@IAMDONMEGA1017       3031 
#DOWNLOAD             3023 
#SPOTIFY              3023 
#ALBUM                3023 
#news                 2851 
#YOUNGMA              2665 
#Hillary              2646 
#LGBT                 2489 
@FBI                  2450 
@newtgingrich         2418 
@seanhannity          2388 
@wikileaks            2304 
#ImWithHer            2149 
@KellyannePolls       2110 
                      ...  
@alanctgardner        1    
@MariEastman          1    
@Ddwyer14             1    
@DeliaandDobie        1    
#amazingexperience  

In [12]:
# Vectorize Text Data 
import sklearn.feature_extraction.text as sk_text


vectorizer = sk_text.TfidfVectorizer(min_df=50,max_features=5000)

matrix = vectorizer.fit_transform(merge_tweets.Text.values)
print(type(matrix))         
tfidf_data = matrix.toarray() 
print(tfidf_data)

<class 'scipy.sparse.csr.csr_matrix'>
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.02394785 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [13]:
# Vectorize Location Data
import sklearn.feature_extraction.text as sk_text

vectorizer2 = sk_text.TfidfVectorizer(min_df=10,max_features=500,stop_words = 'english')

matrix2 = vectorizer2.fit_transform(merge_tweets.Location.values)
print(type(matrix2))         
tfidf_data2 = matrix2.toarray() 
print(tfidf_data2)

<class 'scipy.sparse.csr.csr_matrix'>
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [14]:
#Append back the Tfidf and features from Vectorizer to DataFrame
count_vect_df = pd.DataFrame(matrix.todense(), columns=vectorizer.get_feature_names())

merge_tweets1 = pd.concat([merge_tweets, count_vect_df], axis=1)

count_vect_df2 = pd.DataFrame(matrix2.todense(), columns=vectorizer2.get_feature_names())

merge_tweets = pd.concat([merge_tweets1, count_vect_df2], axis=1)

In [15]:
# Assign X to Location and Text
X = merge_tweets.drop(columns=['UserID','Text','Location','Supporter'])
X.head()

,0hour1__,10,2016,2a,8216,8217,8230,_makada_,abc,abcnews,...,wisconsin,woods,world,worldwide,worth,wv,www,york,yorkshire,zealand
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#Assigne Y as Ground truth 
y = merge_tweets['Supporter'].values
print(y)

[1 1 1 ... 1 1 1]


In [17]:
#Split our Data

from sklearn.model_selection import train_test_split
#split dataset into train and test data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [18]:
#Decision Tree
from sklearn import tree
dtree = tree.DecisionTreeClassifier()

dtree = dtree.fit(X_train, y_train)
y_pred = dtree.predict(X_test)

print(metrics.precision_score(y_test, y_pred, average= 'weighted'))
print(metrics.recall_score(y_test, y_pred, average= 'weighted'))
print(metrics.f1_score(y_test, y_pred, average= 'weighted'))
print(metrics.confusion_matrix(y_test, y_pred, labels=[0, 1]))

0.6463028691837249
0.6428388419164746
0.6438157665597682
[[1100  630]
 [ 764 1409]]


In [47]:
#SVM
# train a SVM model on the training set
#Takes a lot of time to run. and get error like UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
from sklearn.svm import SVC

#probability : boolean, optional (default=False)  Whether to enable probability estimates

clf = SVC(probability=False, C=1.0, gamma='auto')


clf.fit(X_train, y_train) 
y_pred = clf.predict(X_test)

print(metrics.precision_score(y_test, y_pred, average= 'weighted'))
print(metrics.recall_score(y_test, y_pred, average= 'weighted'))
print(metrics.f1_score(y_test, y_pred, average= 'weighted'))
print(metrics.confusion_matrix(y_test, y_pred, labels=[0, 1]))

0.3183006508917341
0.5641813989239047
0.406986876471888
[[   0 1701]
 [   0 2202]]


C:\Jupyter notebook\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Jupyter notebook\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [19]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression

# set a few optional parameter
lr = LogisticRegression(solver='lbfgs', max_iter=300, multi_class='auto')      
lr.fit(X_train, y_train) 
y_pred = lr.predict(X_test)

print(metrics.precision_score(y_test, y_pred, average= 'weighted'))
print(metrics.recall_score(y_test, y_pred, average= 'weighted'))
print(metrics.f1_score(y_test, y_pred, average= 'weighted'))
print(metrics.confusion_matrix(y_test, y_pred))


0.7196579963880919
0.719702792723546
0.7158455021944171
[[1043  687]
 [ 407 1766]]


In [20]:
#Neural Networks

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

scaler = StandardScaler()

scaler.fit(X_train)

X_train= scaler.transform(X_train)

X_test = scaler.transform(X_test)

mlp = MLPClassifier(hidden_layer_sizes=(30,30,30), solver='adam', max_iter=1000)

mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

confusion = metrics.confusion_matrix(y_test, y_pred, labels=[0, 1])

print(confusion)
print(metrics.precision_score(y_test, y_pred, average= 'weighted'))
print(metrics.recall_score(y_test, y_pred, average= 'weighted'))
print(metrics.f1_score(y_test, y_pred, average= 'weighted'))

[[1089  641]
 [ 649 1524]]
0.6696435435685713
0.6694850115295926
0.6695614658032093


In [ ]:
#KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection  import cross_val_score
from sklearn.model_selection import GridSearchCV

In [ ]:
%%time

# Create KNN classifier

k_range = list(range(1, 20))

param_grid = dict(n_neighbors=k_range)
print(param_grid)


knn = KNeighborsClassifier(n_neighbors = 1)

grid = GridSearchCV(knn, param_grid, cv=5, scoring='f1_weighted',n_jobs=2)

# Fit the classifier to the data
grid.fit(X, y)


{'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}


In [ ]:
means = grid.cv_results_['mean_test_score']

for mean, params in zip(means, grid.cv_results_['params']):
    print(mean, params)

In [ ]:
# plot the results
plt.plot(k_range, means)
plt.xlabel('Value of K for KNN')
plt.ylabel('F1 score based on Cross-Validation')
plt.show()

In [ ]:
# identify the best model

print(grid.best_score_)
print(grid.best_params_)
print(grid.best_estimator_)